# <a href="https://colab.research.google.com/github/jupyteronline/notebooks/blob/master/6_nlp/01_자연어처리_기초(IMDB_DATASET).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝을 위한 자연어 기초  
  
딥러닝을 활용하여 텍스트를 분류하기 위해서는  
**1) 토큰화, 2) 단어를 숫자로 변환하는 넘버링, 3) 1차원의 단어를 고차원으로 임베딩, 4) RNN이나 LSTM 같은 모형을 적용**하여 분류하는 4단계 과정을 거치게 됩니다,  
  
  

- **(토큰화)** 토큰화는 연속된 문장을 단어 단위로 잘라주는 것입니다.  
  


- **(넘버링)** 단어마다 순서를 정해 주면 다루기 상당히 편합니다.  
  


- **(임베딩)** 순서가 부여된 단어들은 컴퓨터 입장에서는 연관성을 알 수 없는 숫자들에 불과하다. 따라서 여러 문장을 훈련하여 단어간의 연관성을 부여해준다. 이 과정을 임베딩이라고 한다. 넘버링된 한 개의 숫자가 여러개의 숫자로 벡터화되게 됩니다.  
  
- **(RNN 적용)** 토큰화되고, 토큰화된 단어들이 순서가 부여된 후 임베딩된 문장들은 문장 하나 하나가 RNN 모형에 들어가서 훈련되게 되며, 최종적으로 문장을 분류하는 AI가 만들어지게 됩니다.

    본 튜토리얼에서는 자연어 처리의 기초를 배울 겸 imdb에 있는 영화 리뷰를 바탕으로 감성분석을 해보려고 합니다.

![Imgur](https://i.imgur.com/YFiEvOV.jpg)  
  


![Imgur](https://i.imgur.com/9Z2HwRx.jpg)  
  


![Imgur](https://i.imgur.com/ZABQd8V.jpg)  
  
  

![Imgur](https://i.imgur.com/70HAAbF.jpg)  

In [ ]:
%tensorflow_version 1.x
# IMDB 데이터셋 로드
from keras.datasets import imdb

# Numpy, Pandas, Matplotlib 로드
import numpy as np # 파이썬에서 수치를 다루기 위한 모듈
import pandas as pd # 파이썬에서 table을 다루기 위한 모듈
import matplotlib.pyplot as plt # 파이썬에서 그림을 그리기 위한 모듈
import tensorflow as tf
import os
%matplotlib inline

In [ ]:
import warnings
import tensorflow as tf
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)

imdb 데이터셋을 로드하도록 하겠습니다.  
num_words = 20000은, 문장에 등장하는 단어들의 집합에서, 단어 등장 빈도수가 상위 20,000번째까지만 불러옵니다  
딥러닝 훈련에 쓸 train 데이터와 훈련된 딥러닝의 성능을 검증할 test 데이터를 불러옵니다.

In [ ]:
help(imdb)

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

imdb 데이터셋에서 사용되는 단어와 그 단어에 해당하는 숫자가 맵핑되어 있는 딕셔너리를 불러옵니다. {단어:숫자, 단어:숫자,.....}  
또한 reverse_word_dict를 정의하여 {숫자:단어, 숫자:단어, ...} 형식으로도 만들어줍니다. 


In [ ]:
word_dict = imdb.get_word_index()
reverse_word_dict = {k:v for v, k in word_dict.items()}

In [ ]:
word_dict

단어(이미 토큰화되어있음)와 숫자가 서로 매핑된 것을 볼 수 있다.  
이번 데이터셋은 별도의 토큰화 과정을 거치지 않도록 전처리가 되어 있다.

In [ ]:
for i in x_train[0]:
    
    print(reverse_word_dict[i], end = " ")
print("\n")
print(x_train[0])

In [ ]:
for sent in x_train[0:50]:
  for i in sent:
    
      print(reverse_word_dict[i], end = " ")
  print("\n")
  print(x_train[0])
  print("\n")

문장의 최대 길이, 평균 길이가 얼마나 되는지 알아본다  
문장의 최대 길이는 2494, 평균 길이는 238 정도임을 알 수 있다

In [ ]:
print('maximun length : {}'.format(max(len(l) for l in x_train)))
print('average length : {}'.format(sum(map(len, x_train))/len(x_train)))

plt.hist([len(s) for s in x_train], bins=50)
plt.xlabel('length')
plt.ylabel('number')
plt.show()

vocab_size는 imdb 데이터가 가지고 있는 단어의 개수이다.  
num_words를 20,000으로 했기 때문에 총 단어 개수는 20,000개이다.

In [ ]:
vocab_size = 20000

imdb 데이터셋은 영화 리뷰가 positive인지, negative인지만 알면 된다.  
0인지 1인지만 구분하면 되는 binary classification 문제이다.


In [ ]:
unique, bins = np.unique(y_train, return_counts=True)
print(unique)
print(bins)

케라스를 활용하기 위해서, 필요한 모듈들을 로드한다.  


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Flatten, Dropout, GRU
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

문장의 최대 길이는 2494, 평균 길이는 238 정도였다.  
max_len 변수를 지정해서 문장의 최대 길이를 500으로 맞춰 주도록 하겠다.  
만약 500보다 긴 문장은 문장이 절단될 것이며, 500보다 짧은 문장은 길이를 500으로 맞추기 위해 0으로 채워지게 된다.

In [ ]:
max_len = 500
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

훈련할 모델을 정의한다.
LSTM 대신에 LSTM과 성능은 비슷하면서도 속도는 빠른 GRU 모델을 사용하겠습니다.  
이진 분류이기 때문에 binary_crossentropy 방법으로 loss를 정의하고, adam optimization 방식으로 훈련을 시작하겠습니다.  
Embedding(vocab_size, 100) 이라는 레이어가 있는데, 우리가 imdb 데이터에 사용되는 총 20,000개의 문장을 단어 하나 하나마다 100차원의 임베딩하겠다는 레이어를 뜻합니다.

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(100))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, batch_size=1024, epochs=3, validation_data=(x_test, y_test))

훈련 과정을 그래프로 나타내어 봅니다

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('Training')
plt.ylabel('acc')
plt.xlabel('epochs')
plt.legend(['train', 'val'], loc='upper left')
plt.grid()
plt.show()

F1 SCORE 확인하기

In [ ]:
import sklearn.metrics as metrics

preds = model.predict(x_test)

In [ ]:
print(preds)

In [ ]:
np.round(np.ravel(preds),0), y_test

In [ ]:
print(metrics.classification_report(y_test, np.round(np.ravel(preds),0)))